In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Python inclass practice 7: PDF, CDF, Extreme events

### In this practice, we will continue using the data from last lab
Site id: 04216000 (Niagara River near Buffalo NY) </br>
Site id: 04215000 (Cayuga Creek near Lancaster NY) </br>

## 0. Prep: read in the data

In [ ]:
# just in case you did not get through the first section
df_concat_sel = pd.read_csv("../data/python_inclass_practice_6.data.csv",index_col=0,parse_dates=True)

# 1. PDF & CDF

## 1.1. Probability Density Function

For discrete sample data, we usually use histogram `plt.hist` to visualize it. </br>
Note: this plotting function has an option called `density`. If `density=False`, the plot will show the count falling within each bin. If `density=True`, the plot will show the probability density.

In [ ]:
fig,ax = plt.subplots(1, 1,figsize=[5,4],dpi=200)
ax.hist(df_concat_sel['04216000'],bins=np.arange(100000,350000,10000),density=True)
ax.set_title("PDF for streamflow in Niagara River")

### 1.1.1. It is important to choose bin size and scales wisely

For example, the streamflow regime in Cayuga River is very spiky. Most of the time, the flow is between 0-100 cfs, but it also had some very high flow days with thousands cfs. Therefore, if we use normal scale plots with fixed bins, it will look like left plot below. However, if we use log-scale for x-axis with smart selection of bin width, the plot looks much better (like the one on the right).

In [ ]:
fig,axes = plt.subplots(1, 2,figsize=[10,4],dpi=200)

axes[0].hist(df_concat_sel['04215000'],bins=np.arange(100,4000,100),
             density=True)
axes[1].hist(df_concat_sel['04215000'],bins=np.concatenate([[0,1,2,5,10,20,50,80],
                                                            np.arange(100,4000,100)]),
             density=True)
axes[1].set_xscale('log')
axes[0].set_title("Linear-scale")
axes[1].set_title("Log-scale")

fig.suptitle("PDF for streamflow in Cayuga Creek")

## 1.2. Cumulative Distribution Function

If we have a discrete array of samples, and we would like to know the CDF of the sample, then we can just sort the array. If we look at the sorted result, we'll realize that the smallest value represents 0% , and largest value represents 100 %. Then all other arrays are assigned with evenly distributed probabilities between 0% and 100%.

In [ ]:
# Calculate the cumulative proportion of the data that falls below each value
cumulative = np.linspace(0, 1, len(df_concat_sel['04216000']))

# Sort the data in ascending order
sorted_data = np.sort(df_concat_sel['04216000'])

# Calculate the cumulative proportion of the sorted data
cumulative_data = np.cumsum(sorted_data) / np.sum(sorted_data)

In [ ]:
# Plot the CDF
plt.figure(dpi=300)
plt.plot(sorted_data, cumulative_data, label='streamflow')
plt.ylim(0,1)

# Add labels and title
plt.xlabel("Streamflow (cfs)")
plt.ylabel("Cumulative Proportion")
plt.title("Cumulative Distribution Function (CDF) of Mean daily flow")
plt.legend()
plt.show()

## 1.2.1. Practice #1: plot CDF for Cayuga River. 

In [ ]:
# Please insert your code here

## 1.3. 100-year return floods

As we discussed in class, to calculate the 100-year return floods, we can follow the three steps:
1. Identify annual peak flow `groupby`
2. Calculate the CDF for annual peak flow
3. Calculate the Annual exceedance probability (AEP)
4. In the AEP, find the flow value corresponds to AEP = 1%.

#### 1.3.1. Step 1: Identify annual peak flow

In [ ]:
# Step 1: Identify annual peak flow
df_annual_max = df_concat_sel.groupby(df_concat_sel.index.year).max()
df_annual_max

#### 1.3.2. calculate cdf

In [ ]:
# 2: calculate cdf
# Calculate the cumulative proportion of the data that falls below each value
cumulative_n = np.linspace(0, 1, len(df_annual_max['04216000']))

# Sort the data in ascending order
sorted_data_n = np.sort(df_annual_max['04216000'])

# Calculate the cumulative proportion of the sorted data
cumulative_data_n = np.cumsum(sorted_data_n) / np.sum(sorted_data_n)

In [ ]:
cumulative_data_n

#### 1.3.3. calculate AEP

In [ ]:
# 3: calculate AEP
aep_n = 1 - cumulative_data_n

In [ ]:
# visualize AEP
plt.figure(dpi=200)
plt.plot(aep_n, sorted_data_n,  label='Niagara River Peak Flow')
plt.scatter(aep_n, sorted_data_n)
plt.axvline(0.01,lw=0.5,c='orange')
plt.xlim(0,1)
aep_list = np.array([0.01,0.05,0.1,0.2,0.5,1])
aep_str = []
for i in aep_list:
    if i*100<1:
        aep_str.append("%s%%"%(i*100))
    else:
        aep_str.append("%i%%"%(i*100))
plt.xticks(aep_list,aep_str)
plt.grid(lw=0.4)

# Add labels and title
plt.ylabel("Streamflow (cfs)")
plt.xlabel("Annual Exceedance Probability")
plt.title("Annual Exceedance Probability")
plt.legend()
plt.show()

#### 1.3.4. find the flow corresponds to AEP = 0.01

In [ ]:
# 4: find the flow corresponds to AEP = 0.01

# Basically, we need to find the y value that 
# corresponds to the intersection of orange line 
# and blue dotted line above

# asp_n[::-1] reverse this array, for example
# a = [0,1,2], and a[::-1] will output [2,1,0]

# The reason behind reverse the asp_n is
# np.interp requires an monotonically increasing 
# sample points but our asp_n was in monotonically
# decreasing order.

flow_100yr_flood_n = np.interp(0.01, aep_n[::-1], sorted_data_n[::-1])
print("The flood with 100-year recurrence interval is %.2f cfs"%(flow_100yr_flood_n))

## 1.4.1. Practice #2: Please output the 100-year flood for Cayuga River

In [ ]:
# Insert your code here

## 1.5. 7Q10 for evaluating low flows!
### 1.5.1. We first need to calculate the minimum annual 7-day average flow
The syntax for rolling average is quite simple! </br>
We can use `df.rolling()` to calculate the rolling average. </br>

In [ ]:
df_rolling_7day_mean = df_concat_sel.rolling(7,center=True).mean()

In [ ]:
df_rolling_7day_mean

# Note: the first three days and last three days does not have data!
Because data was not available for the nearby 7-days at those dates.

### 1.5.2. Here we simply drop the NAN in the moving average

In [ ]:
df_rolling_7day_mean = df_rolling_7day_mean.dropna()

### 1.5.3. Practice #3: Can you calculate the 7Q10 for Niagara River?
This is definitely a harder exercise! However, the philosophy behind the calculation is very similar to how we calculate the annual peak flow with 100-year recurrence interval.

7Q10 is the 7-day average annual low flow with 10-year recurrence interval.

1. Calculate the 7-day average flow (we have done in **1.5.1/1.5.2**)
2. Find the annual minimum 7-day average flow (use groupby, see Section **1.3.1**)
3. Calculate CDF, which equals to Annual Non-exceedance Probability (ANEP)!!
4. Find the flow corresponds to ANEP = 0.1 (which corresponds to 10-year recurrence interval)

In [ ]:
# Step 1: Identify annual 7-day low flow


In [ ]:
# Step 2: calculate cdf, which equals to anep


In [ ]:
# Step 3 (optional): visualize ANEP


In [ ]:
# Step 4: find the flow corresponds to ANEP = 0.1
